In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from batching import DataBatcher
from plumbing import load_dataset, load_word_embeddings, dataset_preprocess

project_root = os.path.dirname(os.getcwd())


parameters = {
    "runs_dir": os.path.join(project_root, 'runs'),
    "dataset_directory": os.path.join(project_root, 'snli_1.0'),
    "embeddings_path": os.path.join(project_root, 'GNews-Vectors.bin'),
    "model_name": "attention_lstm",
}

batch_parameters = {
    "batch_size_train": 24,
    "batch_size_dev": 10000,
    "batch_size_test": 10000
}

training_parameters = {
    "learning_rate": 0.001,
    "weight_decay": 0.0,
    "keep_prob": 0.8,
    "batch_size": {"train": batch_parameters['batch_size_train'], 
		   "dev": batch_parameters['batch_size_dev'],
	 	   "test": batch_parameters['batch_size_test']
		  },
    "gpu": 0,  # set to empty string to use CPU
    "num_epochs": 45,
    "embedding_dim": 300,
    "sequence_length": 20,
    "num_units": 100  # LSTM output dimension (k in the original paper)
}

parameters.update(training_parameters)

modeldir = os.path.join(parameters["runs_dir"], parameters["model_name"])
datadir = os.path.join(project_root,"data")

def load_model(embeddings,dataset, setToRunName):
    """
    Load the Model saved during the training (using saver.save)
    """

     
    # Then Define the path
    trained_checkpoint_prefix = os.path.join(project_root, 'runs/attention_lstm/save-1025000')
   
    # Create a Blank graph
    loaded_graph = tf.Graph()
    
    # Create the session
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    config_proto = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)
    with tf.Session(config=config_proto, graph=loaded_graph) as sess:
        # Restore from checkpoint
        # Load the graph from the .meta file
        loader = tf.train.import_meta_graph(trained_checkpoint_prefix + '.meta')
        # Then import all the session variable
        loader.restore(sess, trained_checkpoint_prefix)
        
        # Now the input required tf_placeholder. 
        # They are already defined in the graph.
        # We need to access the reference
        targets_ph = loaded_graph.get_tensor_by_name('targets:0')
        premises_ph = loaded_graph.get_tensor_by_name('premises:0')
        hypothesis_ph = loaded_graph.get_tensor_by_name('hypothesis:0')
        keep_prob_ph = loaded_graph.get_tensor_by_name("keep_prob:0")
        
        #Generate the batcher with the word2vec
        batcher = DataBatcher(embeddings)
    

        # Generate the batches
        batches = batcher.batch_generator(dataset=dataset[setToRunName], num_epochs=1, batch_size=100, seq_length=parameters["sequence_length"])
        for test_step, (batch, _) in enumerate(batches):
            # Generate the input
            feed_dict = {
                premises_ph: np.transpose(batch["premises"], (1, 0, 2)),
                hypothesis_ph: np.transpose(batch["hypothesis"], (1, 0, 2)),
                targets_ph: batch["targets"],
                keep_prob_ph: 1.,
            }

            # Get the accuracy Mean operation
            loss = loaded_graph.get_tensor_by_name('loss/Mean:0')
            accuracy = loaded_graph.get_tensor_by_name('accuracy/Mean:0')
            test_op = loaded_graph.get_tensor_by_name('Merge/MergeSummary:0')
            _ , run_accuracy, run_loss= sess.run([test_op,accuracy, loss], feed_dict=feed_dict)

            # Print the result
            print("\nTEST |  accuracy={0:.2f}%   ".format( 100. * run_accuracy))
        

def load_custom_data(datapath):
    dataset_dir = os.path.join(project_root,'data')
    print("Loading custom test data")
    dataset = {}
    split = "custom"
    df = pd.read_csv(datapath, delimiter='\t')
    dataset[split] = {
        "premises": df[["sentence1"]].values,
        "hypothesis": df[["sentence2"]].values,
        "targets": df[["gold_label"]].values
    }
    return dataset
    
if __name__ == '__main__':
    
    #@ Load the data that we want to test
    dataset_raw = load_custom_data(os.path.join(datadir,"comparisons.txt"))
    dataset = dataset_preprocess(dataset_raw)
    
    # First we need to reload the word 2 vec
    embeddings = load_word_embeddings(parameters['embeddings_path'])
    print("Loaded!")
    
    evalModelOnDataset = load_model(embeddings, dataset, 'custom')


        


Loading custom test data
Loading Word Embeddings
Loaded!
INFO:tensorflow:Restoring parameters from /usr/src/app/runs/attention_lstm/save-1025000

TEST |  accuracy=25.00%   


In [ ]:
from plumbing import load_dataset, load_word_embeddings, dataset_preprocess

# Load SNLI DATASET
snli_dataset = dataset_preprocess(load_dataset(parameters['dataset_directory']))

# Eval
evalModelOnDataset = load_model(embeddings, snli_dataset, 'train')

Loading SNLI dataset
INFO:tensorflow:Restoring parameters from /usr/src/app/runs/attention_lstm/save-1025000

TEST |  accuracy=69.00%   

TEST |  accuracy=61.00%   

TEST |  accuracy=59.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=66.00%   

TEST |  accuracy=62.00%   

TEST |  accuracy=67.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=63.00%   

TEST |  accuracy=55.00%   

TEST |  accuracy=70.00%   

TEST |  accuracy=72.00%   

TEST |  accuracy=59.00%   

TEST |  accuracy=68.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=75.00%   

TEST |  accuracy=76.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=71.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=61.00%   

TEST |  accuracy=63.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=73.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=67.00%   

TEST |  accuracy=67.00


TEST |  accuracy=66.00%   

TEST |  accuracy=68.00%   

TEST |  accuracy=68.00%   

TEST |  accuracy=57.00%   

TEST |  accuracy=71.00%   

TEST |  accuracy=67.00%   

TEST |  accuracy=58.00%   

TEST |  accuracy=74.00%   

TEST |  accuracy=70.00%   

TEST |  accuracy=69.00%   

TEST |  accuracy=62.00%   

TEST |  accuracy=63.00%   

TEST |  accuracy=62.00%   

TEST |  accuracy=68.00%   

TEST |  accuracy=66.00%   

TEST |  accuracy=61.00%   

TEST |  accuracy=60.00%   

TEST |  accuracy=67.00%   

TEST |  accuracy=67.00%   

TEST |  accuracy=60.00%   

TEST |  accuracy=67.00%   

TEST |  accuracy=60.00%   

TEST |  accuracy=60.00%   

TEST |  accuracy=69.00%   

TEST |  accuracy=66.00%   

TEST |  accuracy=58.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=69.00%   

TEST |  accuracy=68.00%   

TEST |  accuracy=67.00%   

TEST |  accuracy=65.00%   

TEST |  accuracy=63.00%   

TEST |  accuracy=64.00%   

TEST |  accuracy=63.00%   

TEST |  accuracy=57